In [142]:
import pickle
import os, sys
import pandas as pd
from tqdm import tqdm
os.chdir('/home/baizhiwang/gki_icd/')
sys.path[0]='../'
!pwd
from src.utils import write_pickle, read_pickle

/home/baizhiwang/gki_icd


获取原始数据

In [104]:
note_events = pd.read_csv("data/mimic4/discharge.csv")

In [4]:
note_events.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [105]:
hadm2subject = {}
for i,row in note_events.iterrows():
    hadm = row['hadm_id']
    subject_id = row['subject_id']
    if hadm not in hadm2subject:
        hadm2subject[hadm] = subject_id

获取数据集划分：subject_id, hadm_id, label，并找到原文本

涉及的HADM_ID

In [13]:
hadm_ids = []
for split in ["train", "dev", "test"]:
    old_df = pd.read_csv(f"data/splits/mimic4_icd9/{split}_full_hadm_ids.csv",header=None)
    hadm_ids.extend(old_df[0])
    # subject_id.extend(old_df["SUBJECT_ID"])
    print(split, len(old_df))

train 188533
dev 7110
test 13709


In [22]:
len(hadm_ids), len(note_events)

(209352, 331793)

在MIMIC-IV v2.2中NoteEvent中找到每个hadm_id对应的报告文本

In [23]:
idx2report = {hadm_id: [] for hadm_id in hadm_ids}
for i, row in tqdm(note_events.iterrows()):
    hadm_id, note_type, note_seq, text = row["hadm_id"], row["note_type"], row["note_seq"], row["text"]
    if hadm_id in hadm_ids and note_type == "DS":
        idx2report[hadm_id].append(text)

331793it [03:03, 1807.25it/s]


查找ICD Code

In [24]:
def reformat(code, is_diag):
    """
    Put a period in the right place because the MIMIC-3 data files exclude them.
    Generally, procedure codes have dots after the first two digits,
    while diagnosis codes have dots after the first three digits.
    """
    code = "".join(code.split("."))
    if is_diag:
        if code.startswith("E"):
            if len(code) > 4:
                code = code[:4] + "." + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + "." + code[3:]
    else:
        code = code[:2] + "." + code[2:]
    return code

In [25]:
dfproc = pd.read_csv(f"data/mimic4/procedures_icd.csv", dtype={"icd_code": str})
dfdiag = pd.read_csv(f"data/mimic4/diagnoses_icd.csv", dtype={"icd_code": str})

ICD9

In [81]:
dfproc9 = dfproc[dfproc["icd_version"] == 9]
dfdiag9 = dfdiag[dfdiag["icd_version"] == 9]

In [27]:
dfdiag9["absolute_code"] = dfdiag9.apply(
    lambda row: str(reformat(str(row[3]), True)), axis=1
)
dfproc9["absolute_code"] = dfproc9.apply(
    lambda row: str(reformat(str(row[4]), False)), axis=1
)

/tmp/ipykernel_3243926/2857917965.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda row: str(reformat(str(row[3]), True)), axis=1
/tmp/ipykernel_3243926/2857917965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfdiag9["absolute_code"] = dfdiag9.apply(
/tmp/ipykernel_3243926/2857917965.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda row: str(reformat(st

In [28]:
dfcodes9 = pd.concat([dfdiag9, dfproc9])

In [32]:
hadm2icd = {hadm_id: [] for hadm_id in hadm_ids}
for i, row in tqdm(dfcodes9.iterrows()):
    hadm_id, icd_code = row["hadm_id"], row["absolute_code"]
    if hadm_id in hadm2icd:
        hadm2icd[hadm_id].append(icd_code)

3377950it [02:00, 28137.90it/s]


MIMIC-IV ICD9 Full

In [ ]:
for split in ["train", "dev", "test"]:
    split_df = pd.read_csv(
        f"data/splits/mimic4_icd9/{split}_full_hadm_ids.csv", header=None
    )
    split_ids = split_df[0].tolist()
    new_df = pd.DataFrame(columns=["subject_id","hadm_id","text","label"])
    samples = []
    for hadm_id in split_ids:
        reports = idx2report[hadm_id]
        text = "\n".join(reports)
        label = hadm2icd[hadm_id]
        subject_id = hadm2subject[hadm_id]
        # csv: Series
        new_df.loc[i] = pd.Series(
            {"subject_id": subject_id, "hadm_id": hadm_id, "text": text,"label":label}
        )
        # pickle: Dict
        sample = {}
        sample["subject_id"] = subject_id
        sample["hadm_id"] = hadm_id
        sample["text"] = text
        sample["label"] = label
        samples.append(sample)
    new_df.to_csv(f"data/mimic4_icd9/{split}.csv", index=False)
    write_pickle(samples, f"data/mimic4_icd9/{split}.pkl")

Code和相应的Description

In [47]:
mimic4_icd9_codes = set()
for icd_codes in hadm2icd.values():
    mimic4_icd9_codes.update(icd_codes)
mimic4_icd9_codes = sorted(mimic4_icd9_codes)
len(mimic4_icd9_codes)

11331

In [63]:
icd9_desc = pd.read_csv("data/icd9/reference/ICD9_descriptions.tsv", sep="\t")
code2desc = dict(zip(icd9_desc["code"], icd9_desc["desc"]))

In [68]:
from random import shuffle

def byLineReader(filename):
    with open(filename, "r", encoding="utf-8") as f:
        line = f.readline()
        while line:
            yield line
            line = f.readline()
    return


class UMLS(object):
    def __init__(
        self, umls_path, source_range=None, lang_range=["ENG"], only_load_dict=False
    ):
        self.umls_path = umls_path
        self.source_range = source_range
        self.lang_range = lang_range
        self.detect_type()
        self.load()
        # if not only_load_dict:
        #     self.load_rel()
        #     self.load_sty()

    def detect_type(self):
        if os.path.exists(os.path.join(self.umls_path, "MRCONSO.RRF")):
            self.type = "RRF"
        else:
            self.type = "txt"

    def load(self):
        reader = byLineReader(os.path.join(self.umls_path, "MRCONSO." + self.type))

        self.cui2str = {}
        self.str2cui = {}
        self.icd9_to_cui = {}
        self.icd10_to_cui = {}
        # self.lui_status = {}
        read_count = 0
        for line in tqdm(reader, ascii=True):
            if self.type == "txt":
                l = [t.replace('"', "") for t in line.split(",")]
            else:
                l = line.strip().split("|")
            cui = l[0]
            lang = l[1]
            # lui_status = l[2].lower() # p -> preferred
            lui = l[3]
            source = l[11]
            code = l[13]
            string = l[14]

            if source == "ICD9CM":
                self.icd9_to_cui[code] = cui

            if source == "ICD10CM" or source == "ICD10PCS":
                self.icd10_to_cui[code] = cui

            if (self.source_range is None or source in self.source_range) and (
                self.lang_range is None or lang in self.lang_range
            ):
                read_count += 1
                self.str2cui[string] = cui
                self.str2cui[string.lower()] = cui
                clean_string = self.clean(string, clean_bracket=False)
                self.str2cui[clean_string] = cui

                if not cui in self.cui2str:
                    self.cui2str[cui] = set()
                self.cui2str[cui].update([string.lower()])
                self.cui2str[cui].update([clean_string])

            # For debug
            # if read_count > 1000:
            #     break

        self.cui = list(self.cui2str.keys())
        shuffle(self.cui)
        self.cui_count = len(self.cui)

        print("cui count:", self.cui_count)
        print("str2cui count:", len(self.str2cui))
        print("MRCONSO count:", read_count)

    def clean(
        self, term, lower=True, clean_NOS=True, clean_bracket=True, clean_dash=True
    ):
        term = " " + term + " "
        if lower:
            term = term.lower()
        if clean_NOS:
            term = term.replace(" NOS ", " ").replace(" nos ", " ")
        if clean_bracket:
            term = re.sub("\\(.*?\\)", "", term)
        if clean_dash:
            term = term.replace("-", " ")
        term = " ".join([w for w in term.split() if w])
        return term

    def icd9_to_str(self, icd):
        if icd in self.icd9_to_cui:
            cui = self.icd9_to_cui[icd]
            str_list = self.cui2str[cui]
            str_list = [w for w in str_list if len(w.split()) >= 2 or len(w) >= 7]
            return list(str_list)
        return []

    def icd10_to_str(self, icd):
        if icd in self.icd10_to_cui:
            cui = self.icd10_to_cui[icd]
            str_list = self.cui2str[cui]
            str_list = [w for w in str_list if len(w.split()) >= 2 or len(w) >= 7]
            return list(str_list)
        return []

In [69]:
umls = UMLS("/home/baizhiwang/gki_icd/data/umls/2024AB/META/", only_load_dict=True)

16709195it [00:56, 295308.19it/s]


cui count: 3426422
str2cui count: 16360443
MRCONSO count: 10378059


In [ ]:
icd9_desc = pd.DataFrame(columns=["code", "desc"])
for icd in tqdm(mimic4_icd9_codes):
    desc = umls.icd9_to_str(icd)[0]
    icd9_desc.loc[len(icd9_desc)] = pd.Series({"code": icd, "desc": desc})
icd9_desc.to_csv(
    "data/mimic4_icd9/codes.csv", index=False,  sep="\t"
)


100%|██████████| 11331/11331 [00:07<00:00, 1527.17it/s]


MIMIC-IV ICD9_50

In [74]:
top50_codes = pd.read_csv(
    "data/splits/mimic4_icd9/top50_icd9_code_list.txt", header=None
)

In [ ]:
for split in ["train", "dev", "test"]:
    split_df = pd.read_csv(
        f"data/splits/mimic4_icd9/{split}_50_hadm_ids.csv", header=None
    )
    split_ids = split_df[0].tolist()
    new_df = pd.DataFrame(columns=["subject_id", "hadm_id", "text", "label"])
    samples = []
    for hadm_id in split_ids:
        reports = idx2report[hadm_id]
        text = "\n".join(reports)
        label = hadm2icd[hadm_id]
        label = [
            code for code in label if code in top50_codes[0].tolist()
        ]
        subject_id = hadm2subject[hadm_id]
        # csv: Series
        new_df.loc[i] = pd.Series(
            {"subject_id": subject_id, "hadm_id": hadm_id, "text": text, "label": label}
        )
        # pickle: Dict
        sample = {}
        sample["subject_id"] = subject_id
        sample["hadm_id"] = hadm_id
        sample["text"] = text
        sample["label"] = label
        samples.append(sample)
    new_df.to_csv(f"data/mimic4_icd9_50/{split}.csv", index=False)
    write_pickle(samples, f"data/mimic4_icd9_50/{split}.pkl")

ICD-10

In [ ]:
dfproc10 = dfproc[dfproc["icd_version"] == 10]
dfdiag10 = dfdiag[dfdiag["icd_version"] == 10]
dfcodes10 = pd.concat([dfdiag10, dfproc10])

In [89]:
hadm2icd10 = {hadm_id: [] for hadm_id in hadm_ids}
for i, row in tqdm(dfcodes10.iterrows()):
    hadm_id, icd_code = row["hadm_id"], row["icd_code"]
    if hadm_id in hadm2icd10:
        hadm2icd10[hadm_id].append(icd_code)

3846225it [02:07, 30079.42it/s]


In [114]:
hadm_ids = []
for split in ["train", "dev", "test"]:
    split_df = pd.read_csv(
        f"data/splits/mimic4_icd10/{split}_full_hadm_ids.csv", header=None
    )
    hadm_ids.extend(split_df[0])
    print(split, len(split_df))

train 110442
dev 4017
test 7851


In [85]:
idx2report = {hadm_id: [] for hadm_id in hadm_ids}
hadm2subject = {}
for i, row in tqdm(note_events.iterrows()):
    subject_id, hadm_id, note_type, note_seq, text = row["subject_id"], row["hadm_id"], row["note_type"], row["note_seq"], row["text"]
    if hadm_id in hadm_ids and note_type == "DS":
        idx2report[hadm_id].append(text)
        if hadm_id not in hadm2subject:
            hadm2subject[hadm_id] = subject_id

331793it [02:22, 2335.99it/s]


In [ ]:

for split in ["train", "dev", "test"]:
    split_df = pd.read_csv(
        f"data/splits/mimic4_icd10/{split}_full_hadm_ids.csv", header=None
    )
    split_ids = split_df[0].tolist()
    new_df = pd.DataFrame(columns=["subject_id", "hadm_id", "text", "label"])
    samples = []
    for hadm_id in split_ids:
        if hadm_id == 23024122:
            continue
        reports = idx2report[hadm_id]
        text = "\n".join(reports)
        label = hadm2icd10[hadm_id]
        subject_id = hadm2subject[hadm_id]
        # csv: Series
        new_df.loc[i] = pd.Series(
            {"subject_id": subject_id, "hadm_id": hadm_id, "text": text, "label": label}
        )
        # pickle: Dict
        sample = {}
        sample["subject_id"] = subject_id
        sample["hadm_id"] = hadm_id
        sample["text"] = text
        sample["label"] = label
        samples.append(sample)
    new_df.to_csv(f"data/mimic4_icd10/{split}.csv", index=False)
    write_pickle(samples, f"data/mimic4_icd10/{split}.pkl")

MIMIC-IV ICD10 Top50

In [138]:
top50_codes = pd.read_csv(
    "data/splits/mimic4_icd10/top50_icd10_code_list.txt", header=None
)

In [ ]:
for split in ["train", "dev", "test"]:
    split_df = pd.read_csv(
        f"data/splits/mimic4_icd10/{split}_50_hadm_ids.csv", header=None
    )
    split_ids = split_df[0].tolist()
    new_df = pd.DataFrame(columns=["subject_id", "hadm_id", "text", "label"])
    samples = []
    for hadm_id in split_ids:
        if hadm_id == 23024122:
            continue
        reports = idx2report[hadm_id]
        text = "\n".join(reports)
        label = hadm2icd10[hadm_id]
        label = [code for code in label if code in top50_codes[0].tolist()]
        subject_id = hadm2subject[hadm_id]
        # csv: Series
        new_df.loc[i] = pd.Series(
            {"subject_id": subject_id, "hadm_id": hadm_id, "text": text, "label": label}
        )
        # pickle: Dict
        sample = {}
        sample["subject_id"] = subject_id
        sample["hadm_id"] = hadm_id
        sample["text"] = text
        sample["label"] = label
        samples.append(sample)
    new_df.to_csv(f"data/mimic4_icd10_50/{split}.csv", index=False)
    write_pickle(samples, f"data/mimic4_icd10_50/{split}.pkl")

Check Final Files

In [143]:
train_data = read_pickle("data/mimic4_icd9/train.pkl")
len(train_data)

188533

In [145]:
for dic in train_data:
    if len(dic["label"]) == 0:
        print(dic["label"])
        # break

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
